In [2]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sb
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [365]:
class ScratchClfEvaluation():

    
    def __init__():
        pass
    
    
    def _AND_logic(self, x, y):
        tmp = x*0.5 + y*0.5 - 0.7
        return (tmp >= 0).astype(int)


    def _OR_logic(self, x, y):
        tmp = x*0.5 + y*0.5 - 0.2
        return (tmp >= 0).astype(int)


    def _NAND_logic(self, x, y):
        tmp = x*(-0.5) + y*(-0.5) + 0.7
        return (tmp >= 0).astype(int)


    def _XOR_logic(self, x, y):
        tmp1 = self._NAND_logic(x, y)
        tmp2 = self._OR_logic(x, y)
        tmp = self._AND_logic(tmp1, tmp2)
        return tmp

    
    def _cal_tp(self, test_target, result, pos_label=1):
        if pos_label==1:
            x = test_target
            y = result     
        elif pos_label==0:
            x = self._NAND_logic(test_target, test_target)
            y = self._NAND_logic(result, result)       

        tmp = self._AND_logic(x, y)
        return np.sum(tmp == 1)


    def _cal_fp(self, test_target, result, pos_label=1):
        if pos_label==1:
            x = test_target
            y = result     
        elif pos_label==0:
            x = self._NAND_logic(test_target, test_target)
            y = self._NAND_logic(result, result)

        tmp1 = self._XOR_logic(x, y)
        tmp = self._AND_logic(tmp1, y)
        return np.sum(tmp == 1)


    def _cal_tn(self, test_target, result, pos_label=1):
        if pos_label==0:
            x = test_target
            y = result     
        elif pos_label==1:
            x = self._NAND_logic(test_target, test_target)
            y = self._NAND_logic(result, result)

        tmp = self._AND_logic(x, y)
        return np.sum(tmp == 1)


    def _cal_fn(self, test_target, result, pos_label=1):
        if pos_label==1:
            x = test_target
            y = result     
        elif pos_label==0:
            x = self._NAND_logic(test_target, test_target)
            y = self._NAND_logic(result, result)

        tmp1 = self._XOR_logic(x, y)
        tmp2 = self._AND_logic(tmp1, x)
        return np.sum(tmp2 == 1)


    def eval_tpr(self, ans, pred, pos_label=1): 
        sum_of_pos = np.sum(ans == int(pos_label))
        tpr = self._cal_tp(ans, pred, pos_label) / sum_of_pos
        return tpr


    def eval_fpr(self, ans, pred, pos_label=1):
        sum_of_neg = np.sum(ans != int(pos_label))
        fpr = self._cal_fp(ans, pred, pos_label) / sum_of_neg
        return fpr
    
    
    def eval_accuracy(self, ans, pred, pos_label=1):
        numer = self._cal_tp(ans, pred) + self._cal_tn(ans, pred)
        denom = self._cal_tp(ans, pred) + self._cal_tn(ans, pred) + self._cal_fp(ans, pred) + self._cal_fn(ans, pred)
        print("tp={} tn={} fp={} fn={}".format(self._cal_tp(ans, pred), 
                                               self._cal_tn(ans, pred),
                                               self._cal_fp(ans, pred),
                                               self._cal_fn(ans, pred)))
        
        return numer / denom
    
    
    def eval_precision(self, ans, pred, pos_label=1):
        numer = self._cal_tp(ans, pred)
        denom = self._cal_tp(ans, pred) + self._cal_fp(ans, pred)
        
        return numer / denom

    
    def eval_recall(self, ans, pred, pos_label=1):
        numer = self._cal_tp(ans, pred)
        denom = self._cal_tp(ans, pred) + self._cal_fn(ans, pred)
        
        return numer / denom
    
    
    def eval_f1(self, ans, pred, pos_label=1):
        numer = 2 * self.eval_precision(ans, pred) * self.eval_recall(ans, pred)
        denom = self.eval_precision(ans, pred) + self.eval_recall(ans, pred)
        
        return numer / denom

In [3]:
iris_data_set = load_iris()
x = pd.DataFrame(iris_data_set.data, columns=iris_data_set.feature_names)    #Put explanatory variable into x as pandasdata frame
y = pd.DataFrame(iris_data_set.target, columns=['Species'])    #Put iris response variable into y as pandasdata frame
df = pd.concat([x, y], axis=1)

In [4]:
col_name = df.columns.values
data = df[[col_name[0], col_name[2], col_name[4]]]
data = data[data["Species"] != 0]
data.head()

,sepal length (cm),petal length (cm),Species
50,7.0,4.7,1
51,6.4,4.5,1
52,6.9,4.9,1
53,5.5,4.0,1
54,6.5,4.6,1


In [437]:
import numpy as np
import matplotlib.pyplot as plt


class ScratchSVMClassifier(ScratchClfEvaluation):
    """
    線形回帰のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    no_bias : bool
      バイアス項を入れない場合はTrue
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.coef_ : 次の形のndarray, shape (n_features,)
      パラメータ
    self.loss : 次の形のndarray, shape (self.iter,)
      学習用データに対する損失の記録
    self.val_loss : 次の形のndarray, shape (self.iter,)
      検証用データに対する損失の記録

    """

    def __init__(self, num_iter, lr, verbose, C=0, threshold=1e-5, hit_vector_cnt_threshold=5, kernel='linear'):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.C = C
        self.threshold = threshold
        if hit_vector_cnt_threshold >= 2:
            self.hit_vector_cnt_threshold = hit_vector_cnt_threshold
        else:
            self.hit_vector_cnt_threshold = 2
        self.kernel = kernel
        self.verbose = verbose
        # 損失を記録する配列を用意 
        self.lam = 0
        self.sp_vector = 0
        self.num_of_feature = 0
        self.num_of_samples = 0
        self.label0_val = 0
        self.label1_val = 0

    def fit(self, X, y, X_val=None, y_val=None):
        """
        線形回帰を学習する。検証用データが入力された場合はそれに対する損失と精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            学習用データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            学習用データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証用データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証用データの正解値
        """

        # 1-1. データを shape(n_feature, n_samples)へ整形, θをshape(n_feature, 1)へ整形　h=dot( (θ)t, X)とするため。
        train_feature = X.T
        train_target = y.reshape(1, len(y))
        test_feature = X_val.T
        test_target = y_val.reshape(1, len(y_val))
        self.num_of_feature = train_feature.shape[0]
        self.num_of_samples = train_feature.shape[1]
        
        # 1-2. Yを-1 or 1に規格化
        self.label0_val = max(y)
        self.label1_val = min(y)
        train_target[train_target == self.label0_val] = -1
        train_target[train_target == self.label1_val] = 1        
        test_target[test_target == self.label0_val] = -1
        test_target[test_target == self.label1_val] = 1
        
        # 1-3. 準備 サポートベクターの検出のためのFeature+Target行列を用意
        data_source = np.concatenate((train_feature, train_target), axis=0)
        HIT_LABEL_CNT_THRESHOLD = (int)(self.hit_vector_cnt_threshold / 2)
        
        # 1-4. 準備 適当な値でλを定義(random)、サンプル数分準備
        LAMBDA_INIT_MIN = 1
        LAMBDA_INIT_MAX = 10
        LAMBDA_INIT_SCALE = 1e-07
        self.lam = np.random.randint(LAMBDA_INIT_MIN, LAMBDA_INIT_MAX, train_target.shape[1]) * LAMBDA_INIT_SCALE
        self.lam = np.reshape(self.lam, (1, len(self.lam)))
        self.lam_cal_log = np.zeros((1, (len(self.lam))))
        print("Initial lambda:\n{}".format(self.lam))

        # 2. 最急降下法(Loopをiter回だけ回す)　
        for i in range(0, self.iter):
            self.lam = self._gradient_descent(train_feature, train_target)
            if self.hit_vector_cnt_threshold <= np.sum(self.lam > self.threshold):
                #学習データからサポートベクターを抜き出す
                #data_source = np.concatenate([data_source, self.lam], axis=0)
                selecter = self.lam * np.ones((data_source.shape[0], 1))
                sp_vector = data_source[selecter > self.threshold]
                sp_vector = sp_vector.reshape(data_source.shape[0], (int)(len(sp_vector)/data_source.shape[0]))
                #抜き出したサポートベクターが+/-の両方含んでいる場合ループを抜けて計算を終了。サポートベクターをメンバ変数に保存
                label_p_cnt = np.sum([sp_vector[sp_vector.shape[0] - 1] == 1]) 
                label_n_cnt = np.sum([sp_vector[sp_vector.shape[0] - 1] == -1])
                if label_p_cnt >= HIT_LABEL_CNT_THRESHOLD & label_n_cnt >= HIT_LABEL_CNT_THRESHOLD:
                    print("Loop count={}".format(i))
                    print("Support vector=\n{}".format(sp_vector))
                    print("Labe count 1:[{}] -1:[{}]".format(label_p_cnt, label_n_cnt))
                    self.sp_vector = sp_vector
                    self.lam = self.lam[self.lam > self.threshold]
                    print("accuracy:",self.cal_accuracy(train_target, self._test_predict(train_feature)))
                    break
                    
        return

    def predict(self, X):
        """
        線形回帰を使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            線形回帰による推定結果
        """

        # 入力値を整形
        X = X.T

        x_sn = self.sp_vector[0:self.sp_vector.shape[0] - 1]
        x_sn = x_sn.reshape((self.num_of_feature, x_sn.shape[1]))
        y_sn = self.sp_vector[self.sp_vector.shape[0] - 1].reshape((1, x_sn.shape[1]))    
        tmp1 = self._svm_kernel_function(X, x_sn)
        #print("lam=",self.lam.shape)
        #print("y_sn=",y_sn.shape)
        #print("tmp1=",tmp1.shape)
        tmp2 = self.lam.T * y_sn * tmp1
        result = np.sum(tmp2, axis=1)
        result[result < 0] = -1
        result[result >= 0] = 1
        result = result.astype('int8').T
        
        return self._standval_to_actval(result)

    #def cal_confusion_matrix(self, y_pred, y):
        
    def cal_accuracy(self, y_pred, y):
        return self.eval_accuracy(self._actval_to_standval(y_pred), self._actval_to_standval(y))
            
    def cal_precision(self, y_pred, y):
        return self.eval_precision(self._actval_to_standval(y_pred), self._actval_to_standval(y))
        
    def cal_recall(self, y_pred, y):
        return self.eval_recall(self._actval_to_standval(y_pred), self._actval_to_standval(y))
        
    def cal_f1(self, y_pred, y):
        return self.eval_f1(self._actval_to_standval(y_pred), self._actval_to_standval(y))
    
    def _test_predict(self, X):
        """
        SVMを使い推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            線形回帰による推定結果
        """
        
        x_sn = self.sp_vector[0:self.sp_vector.shape[0] - 1]
        x_sn = x_sn.reshape((self.num_of_feature, x_sn.shape[1]))
        y_sn = self.sp_vector[self.sp_vector.shape[0] - 1].reshape((1, x_sn.shape[1]))    
        tmp1 = self._svm_kernel_function(X, x_sn)
        #print("lam=",self.lam.shape)
        #print("y_sn=",y_sn.shape)
        #print("tmp1=",tmp1.shape)        
        tmp2 = self.lam.T * y_sn * tmp1
        result = np.sum(tmp2, axis=1)
        result[result < 0] = -1
        result[result >= 0] = 1
        
        return result.astype('int8').T
    
    def _svm_kernel_function(self, X1, X2):
        """
        SVM kernel関数
        dot(X1(転置), X2)を計算

        Parameters
        ----------
        X : 次の形のndarray, shape (n_features, n_samples)

        Returns
        -------
          次の形のndarray, shape (n_samples, n_samples)
        """
        ans = np.dot(X1.T, X2)
        return ans   

    def _gradient_descent(self, X, y):
        """
        説明を記述
        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
          学習データ
        y : 次の形のndarray, shape (n_samples, 1)
          正解値

        Returns
        -------        
        """

        tmp1 = y.T * y * self.lam * self._svm_kernel_function(X, X)
        delta = 1 - (np.sum(tmp1, axis=0))
        delta = delta.reshape(len(delta), 1)
        result = self.lam + self.lr * delta.T
        #計算の都合で<0の結果は0に置き換える
        result[result < 0] = 0
        
        return result
    
    def _standval_to_actval(self, x):
        tmp = x 
        tmp[tmp == 1] = self.label1_val
        tmp[tmp == -1] = self.label0_val

        return tmp
    
    def _actval_to_standval(self, x):
        tmp = x
        tmp[tmp == self.label0_val] = -1
        tmp[tmp == self.label1_val] = 1
        
        return tmp
        
    def plot_boundary(self, x):
        return
    
    def view_result(self):
        print(self.sp_vector)
        return

In [438]:
clf = ScratchSVMClassifier(1000, 1e-8, False, C=0)

In [439]:
train_feature, test_feature, train_target, test_target = train_test_split(data[['sepal length (cm)', 'petal length (cm)']].values, data['Species'].values, test_size=0.4, random_state=None)

In [440]:
#学習データの特徴量を標準化
scaler = StandardScaler()
scaler.fit(train_feature)
train_feature = scaler.transform(train_feature)

scaler.fit(test_feature)
test_feature = scaler.transform(test_feature)

In [441]:
clf.fit(train_feature, train_target, test_feature, test_target)

Initial lambda:
[[3.e-07 1.e-07 8.e-07 7.e-07 7.e-07 2.e-07 9.e-07 3.e-07 7.e-07 3.e-07
  2.e-07 7.e-07 8.e-07 5.e-07 9.e-07 5.e-07 5.e-07 8.e-07 9.e-07 2.e-07
  9.e-07 7.e-07 9.e-07 1.e-07 2.e-07 8.e-07 3.e-07 2.e-07 8.e-07 9.e-07
  3.e-07 3.e-07 1.e-07 4.e-07 9.e-07 4.e-07 8.e-07 2.e-07 5.e-07 2.e-07
  1.e-07 3.e-07 9.e-07 1.e-07 4.e-07 8.e-07 4.e-07 4.e-07 8.e-07 6.e-07
  3.e-07 8.e-07 5.e-07 5.e-07 2.e-07 3.e-07 3.e-07 9.e-07 8.e-07 8.e-07]]
Loop count=910
Support vector=
[[-0.12844329  0.19266494 -1.25232213  0.35321906 -1.09176801  0.19266494
   0.51377318 -0.12844329  0.35321906]
 [-0.20412415  0.94407417 -0.45927933 -0.71443451 -1.09716728  1.45438453
  -0.33170174 -0.33170174  0.81649658]
 [ 1.         -1.          1.          1.          1.         -1.
   1.          1.         -1.        ]]
Labe count 1:[6] -1:[3]
tp=29 tn=26 fp=0 fn=0
accuracy: 1.0


In [443]:
pred = clf.predict(test_feature)
print(clf.cal_accuracy(pred, test_target))

print(pred)
print(test_target)

tp=16 tn=18 fp=0 fn=0
1.0
[-1  1  1  1  1 -1 -1  1 -1 -1  1  1 -1  1 -1  1 -1  1 -1 -1 -1  1 -1 -1
  1  1 -1 -1  1  1  1  1  1 -1  1 -1  1  1 -1 -1]
[-1  1  1  1  1 -1 -1 -1  1 -1 -1  1 -1 -1 -1  1 -1  1 -1 -1 -1  1 -1 -1
 -1  1 -1 -1  1  1  1  1 -1 -1  1 -1  1  1 -1 -1]
